In [37]:
#Changes data into readable form
import requests #To hit the server with a request to read the data or extract the data
from bs4 import BeautifulSoup #BeautifulSoup is used to extract HTML from inspection session. (Used for scrapping)
import pandas as pd
#PANDA will be used for data analysis
import sqlite3 #JUST A NORMAL VERSION OF SQL LITE IS OPENED
from datetime import datetime #WHICH THING IS RUNNING ON WHICH TIME
# from icecream import ic
import time

### CREATING LOG FILE

In [38]:
def log_progress(message):
    """This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing"""

    with open('./logs/code_log.txt', 'a') as f:
        f.write(f'{datetime.now()}: {message}\n')
        #Creating a file logging each time and date for incoming data

### EXTRACTING 5 CATEGORIES

In [ ]:
# Step 1: Choose 5 categories (you can pick any from Banggood)
categories = {
    'smartphones': 'https://www.banggood.com/search/smartphone.html?last_spm=1a981.Homepage.0001127294.000119935.e2248485d00342a5960ed141694c9965',
    'laptops': 'https://www.banggood.com/search/laptop.html?from=nav', 
    'headphones': 'https://www.banggood.com/search/headphones.html?from=nav',
    'smartwatches': 'https://www.banggood.com/search/smartwatch.html',
    'cameras': 'https://www.banggood.com/search/cameras.html'
}
def scrape_category(category_url, category_name):
    products = []
    
    # Step 2: Go through multiple pages (pagination)
    for page in range(1, 6):  # Scrape first 5 pages
        # Add page number to URL
        url = f"{category_url}?page={page}"
        
        # Step 3: Request the webpage
        response = requests.get(url)
        
        # Step 4: Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Step 5: Find all product containers
        product_cards = soup.find_all('div', class_='product-card')
        
        for card in product_cards:
            try:
                # Extract product name
                name = card.find('a', class_='title').text.strip()
                
                # Extract price
                price = card.find('span', class_='price').text.strip()
                
                # Extract rating
                rating_element = card.find('span', class_='rating')
                rating = rating_element.text.strip() if rating_element else "No rating"
                
                # Extract reviews
                reviews_element = card.find('span', class_='review')
                reviews = reviews_element.text.strip() if reviews_element else "0 reviews"
                
                # Extract product URL
                product_url = card.find('a', class_='title')['href']
                if not product_url.startswith('http'):
                    product_url = 'https:' + product_url
                
                # Add to our list
                products.append({
                    'category': category_name,
                    'name': name,
                    'price': price,
                    'rating': rating,
                    'reviews': reviews,
                    'url': product_url
                })
                
            except Exception as e:
                print(f"Error scraping product: {e}")
                continue
        
        # Be nice to the website - wait between requests
        time.sleep(1)
    
    return products

### CLEANING EXTRACTED DATA

In [40]:
import pandas as pd
import numpy as np

def clean_data(products_list):
    # Step 1: Create a DataFrame (like an Excel spreadsheet)
    df = pd.DataFrame(products_list)
    
    # Step 2: Clean prices (remove $, € symbols and convert to numbers)
    df['price_clean'] = df['price'].str.replace('$', '').str.replace('€', '').str.replace(',', '').astype(float)
    
    # Step 3: Clean ratings (extract numbers only)
    df['rating_clean'] = pd.to_numeric(df['rating'].str.extract('([0-9.]+)')[0], errors='coerce')
    
    # Step 4: Clean reviews (extract numbers only)
    df['reviews_clean'] = pd.to_numeric(df['reviews'].str.extract(r'(\d+)')[0], errors='coerce').fillna(0)
    
    # Step 5: Handle missing values
    df['rating_clean'] = df['rating_clean'].fillna(df['rating_clean'].mean())  # Fill missing ratings with average
    
    # Step 6: Create derived features (new calculated columns)
    
    # Feature 1: Price category (Budget, Mid-range, Premium)
    df['price_category'] = pd.cut(df['price_clean'], 
                                 bins=[0, 50, 200, float('inf')], 
                                 labels=['Budget', 'Mid-range', 'Premium'])
    
    # Feature 2: Value score (rating per dollar spent)
    df['value_score'] = df['rating_clean'] / df['price_clean']
    df['value_score'] = df['value_score'].replace([np.inf, -np.inf], 0)  # Handle division by zero
     
    # Step 7: Remove duplicates
    df = df.drop_duplicates(subset=['name', 'category'])
    
    return df

### Python Exploratory Analysis

In [41]:
import matplotlib.pyplot as plt
import seaborn as sns

def perform_analysis(df):
    # Set up styling for our graphs
    plt.style.use('default')
    sns.set_palette("husl") #For Visually Appealing Colours
    
    # Analysis 1: Price distribution per category
    plt.figure(figsize=(12, 6))
    for category in df['category'].unique():
        category_prices = df[df['category'] == category]['price_clean']
        plt.hist(category_prices, alpha=0.7, label=category, bins=20)
    
    plt.title('Price Distribution by Category')
    plt.xlabel('Price ($)')
    plt.ylabel('Number of Products')
    plt.legend()  #A KEY DESCRIBING FEATURES OF GRAPHS e.g Which colour represents which quantity
    plt.show()
    
    # Analysis 2: Rating vs Price correlation
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df, x='price_clean', y='rating_clean', hue='category')
    plt.title('Rating vs Price Correlation')
    plt.xlabel('Price ($)')
    plt.ylabel('Rating')
    plt.show()
    
    # Analysis 3: Top reviewed products
    top_reviewed = df.nlargest(10, 'reviews_clean')[['name', 'category', 'reviews_clean', 'rating_clean']]
    print("Top 10 Most Reviewed Products:")
    print(top_reviewed)
    
    # Analysis 4: Best value products (high rating, low price)
    best_value = df.nlargest(10, 'value_score')[['name', 'category', 'price_clean', 'rating_clean', 'value_score']]
    print("\nTop 10 Best Value Products:")
    print(best_value)
    
    # Analysis 5: Average price and rating per category
    category_stats = df.groupby('category').agg({
        'price_clean': ['mean', 'min', 'max'],
        'rating_clean': 'mean',
        'name': 'count'
    }).round(2)
    
    print("\nCategory Statistics:")
    print(category_stats)

### DATABASE CONNECTIVITY

In [42]:
import pyodbc

def setup_database():
    # Step 1: Connect to SQL Server
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER=DESKTOP-5AHJ9AS;' 
        'DATABASE=master;'   # Connect to master database first
        'Trusted_Connection=yes;'
    )
    
    cursor = conn.cursor()
    
    # Step 2: Create our database
    cursor.execute("""
        IF NOT EXISTS(SELECT name FROM sys.databases WHERE name = 'BanggoodAnalysis')
        CREATE DATABASE BanggoodAnalysis
    """)
    
    conn.close()

def create_table():
    # Connect to our new database
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER=DESKTOP-5AHJ9AS;'
        'DATABASE=BanggoodAnalysis;'
        'Trusted_Connection=yes;'
    )
    
    cursor = conn.cursor()
    
    # Step 3: Create products table
    cursor.execute("""
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='products' AND xtype='U')
        CREATE TABLE products (
            id INT IDENTITY(1,1) PRIMARY KEY,
            category NVARCHAR(100),
            name NVARCHAR(255),
            price_clean FLOAT,
            rating_clean FLOAT,
            reviews_clean INT,
            price_category NVARCHAR(50),
            value_score FLOAT,
            url NVARCHAR(500),
            scraped_date DATETIME DEFAULT GETDATE()
        )
    """)
    
    conn.commit()
    conn.close()

def insert_data(df):
    # Connect to database
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER=DESKTOP-5AHJ9AS;'
        'DATABASE=BanggoodAnalysis;'
        'Trusted_Connection=yes;'
    )
    
    # Step 4: Insert data row by row
    for index, row in df.iterrows():
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO products (category, name, price_clean, rating_clean, reviews_clean, price_category, value_score, url)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, row['category'], row['name'], row['price_clean'], row['rating_clean'], 
             row['reviews_clean'], row['price_category'], row['value_score'], row['url'])
    
    conn.commit()
    
    # Step 5: Validate - count how many rows we inserted
    cursor.execute("SELECT COUNT(*) FROM products")
    row_count = cursor.fetchone()[0]
    print(f"Successfully inserted {row_count} products!")
    
    conn.close()

### Part 5: SQL Aggregated Analysis (Minimum 5 Queries)


In [43]:
def run_sql_queries():
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER=DESKTOP-5AHJ9AS;'
        'DATABASE=BanggoodAnalysis;'
        'Trusted_Connection=yes;'
    )
    
    # Query 1: Average price per category
    print("1. Average Price per Category:")
    query1 = """
        SELECT category, AVG(price_clean) as avg_price
        FROM products 
        GROUP BY category
        ORDER BY avg_price DESC
    """
    result1 = pd.read_sql(query1, conn)
    print(result1)
    
    # Query 2: Average rating per category  
    print("\n2. Average Rating per Category:")
    query2 = """
        SELECT category, AVG(rating_clean) as avg_rating
        FROM products 
        GROUP BY category
        ORDER BY avg_rating DESC
    """
    result2 = pd.read_sql(query2, conn)
    print(result2)
    
    # Query 3: Product count per category
    print("\n3. Product Count per Category:")
    query3 = """
        SELECT category, COUNT(*) as product_count
        FROM products 
        GROUP BY category
        ORDER BY product_count DESC
    """
    result3 = pd.read_sql(query3, conn)
    print(result3)
    
    # Query 4: Top 5 reviewed items per category
    print("\n4. Top 5 Reviewed Items per Category:")
    query4 = """
        WITH RankedProducts AS (
            SELECT *, 
                   ROW_NUMBER() OVER (PARTITION BY category ORDER BY reviews_clean DESC) as rank
            FROM products
        )
        SELECT category, name, reviews_clean, rating_clean
        FROM RankedProducts 
        WHERE rank <= 5
        ORDER BY category, reviews_clean DESC
    """
    result4 = pd.read_sql(query4, conn)
    print(result4)
    
    # Query 5: Price category distribution
    print("\n5. Price Category Distribution:")
    query5 = """
        SELECT price_category, COUNT(*) as count,
               CAST(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM products) AS DECIMAL(5,2)) as percentage
        FROM products 
        GROUP BY price_category
        ORDER BY count DESC
    """
    result5 = pd.read_sql(query5, conn)
    print(result5)
    
    conn.close()

# ETL

In [ ]:
def generate_report():
    print("=" * 50)
    print("BANGGOOD PRODUCT ANALYSIS - FINAL REPORT")
    print("=" * 50)
    
    # Architecture Summary
    print("\n📊 ARCHITECTURE OVERVIEW:")
    print("1. Data Extraction → Web scraping from Banggood")
    print("2. Data Cleaning → Handle missing values, create features")  
    print("3. Python Analysis → Statistical analysis & visualization")
    print("4. SQL Storage → Structured database storage")
    print("5. SQL Analysis → Advanced queries and aggregations")
    
    # Key Findings
    print("\n🔍 KEY FINDINGS:")
    print("- Price distribution varies significantly across categories")
    print("- Some categories show strong correlation between price and rating")
    print("- Best value products identified using value score metric")
    print("- Popular products tend to have more reviews and higher ratings")
    
    # Recommendations
    print("\n💡 RECOMMENDATIONS:")
    print("1. Focus on high-value products for customer recommendations")
    print("2. Monitor pricing trends in competitive categories")
    print("3. Use review count as an indicator of product popularity")
    print("4. Consider price category segmentation for marketing")

# Main execution function
def main():
    print("Starting Banggood Data Pipeline...")
    
    # Part 1: Scrape data
    all_products = []
    for category_name, category_url in categories.items():
        print(f"Scraping {category_name}...")
        products = scrape_category(category_url, category_name)
        all_products.extend(products)
    
    # Part 2: Clean data
    print("Cleaning data...")
    cleaned_df = clean_data(all_products)
    
    # Part 3: Analyze data
    print("Performing analysis...")
    perform_analysis(cleaned_df)
    
    # Part 4: Database operations
    print("Setting up database...")
    setup_database()
    create_table()
    insert_data(cleaned_df)
    
    # Part 5: SQL analysis
    print("Running SQL queries...")
    run_sql_queries()
    
    # Part 6: Final report
    generate_report()
    
    print("\n✅ Pipeline completed successfully!")

# Run the entire pipeline
if __name__ == "__main__":
    main()
    log_progress('Preliminaries complete. Initiating ETL process')
